## Assignment - Part 1, Querying a Database

In [1]:
import sqlite3
import pandas as pd

How many total characters are there?

In [2]:
conn = sqlite3.connect('rpg_db.sqlite3')
curs = conn.cursor()
query = 'SELECT COUNT(*) FROM charactercreator_character;'
curs.execute(query)
curs.fetchall()

[(302,)]

How many of each specific subclass?

In [3]:
query = 'SELECT COUNT(*) FROM charactercreator_mage;'
curs.execute(query)
curs.fetchall()

[(108,)]

In [4]:
query = 'SELECT COUNT(*) FROM charactercreator_necromancer;'
curs.execute(query)
curs.fetchall()

[(11,)]

In [5]:
query = 'SELECT COUNT(*) FROM charactercreator_thief;'
curs.execute(query)
curs.fetchall()

[(51,)]

In [6]:
query = 'SELECT COUNT(*) FROM charactercreator_cleric;'
curs.execute(query)
curs.fetchall()

[(75,)]

In [7]:
query = 'SELECT COUNT(*) FROM charactercreator_fighter;'
curs.execute(query)
curs.fetchall()

[(68,)]

How many total items?

In [8]:
query = 'SELECT COUNT(*) FROM armory_item;'
curs.execute(query)
curs.fetchall()

[(174,)]

How many of the Items are weapons?

In [9]:
query = 'SELECT COUNT(*) FROM armory_weapon;'
curs.execute(query)
curs.fetchall()

[(37,)]

How many are not?

In [10]:
# where method
query = ('SELECT COUNT(*) FROM armory_item '
         'WHERE item_id NOT IN (SELECT item_ptr_id FROM armory_weapon);')
#OR
query = '''SELECT COUNT(*) FROM armory_item
           WHERE item_id NOT IN (SELECT item_ptr_id FROM armory_weapon);'''

curs.execute(query)
curs.fetchall()

[(137,)]

In [11]:
# join method
query = '''SELECT COUNT(*) FROM armory_item
           LEFT JOIN armory_weapon ON item_id = item_ptr_id
           WHERE item_ptr_id is NULL;'''
curs.execute(query)
curs.fetchall()

[(137,)]

How many Items does each character have? (Return first 20 rows)

In [12]:
query = ''' SELECT character_id, COUNT(item_id) as item_count
            FROM charactercreator_character_inventory
            GROUP BY character_id
            LIMIT 20;'''
curs.execute(query)
curs.fetchall()

# Technically, shouldn't assume all characters have an item, even if it turns out they do,
# see next problem for a more correct, general answer 

[(1, 3),
 (2, 3),
 (3, 2),
 (4, 4),
 (5, 4),
 (6, 1),
 (7, 5),
 (8, 3),
 (9, 4),
 (10, 4),
 (11, 3),
 (12, 3),
 (13, 4),
 (14, 4),
 (15, 4),
 (16, 1),
 (17, 5),
 (18, 5),
 (19, 3),
 (20, 1)]

How many Weapons does each character have? (Return first 20 rows)

In [13]:
query = ''' SELECT ccc.character_id, COUNT(aw.item_ptr_id) as weapon_count
            FROM charactercreator_character as ccc
            LEFT JOIN charactercreator_character_inventory as cci                    
            ON ccc.character_id = cci.character_id
            LEFT JOIN armory_weapon as aw
            ON cci.item_id = aw.item_ptr_id
            GROUP BY ccc.character_id
            LIMIT 20;'''
curs.execute(query)
curs.fetchall()

[(1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 2),
 (6, 0),
 (7, 1),
 (8, 0),
 (9, 0),
 (10, 0),
 (11, 1),
 (12, 0),
 (13, 0),
 (14, 0),
 (15, 0),
 (16, 0),
 (17, 0),
 (18, 0),
 (19, 0),
 (20, 1)]

In [14]:
# Incomplete Method 1 - because we are assuming
# charactercreator_character character_id's all have items

query = ''' SELECT cci.character_id, COUNT(aw.item_ptr_id) as weapon_count
            FROM charactercreator_character_inventory as cci
            LEFT JOIN armory_weapon as aw
            ON cci.item_id = aw.item_ptr_id
            GROUP BY character_id
            LIMIT 20;'''
curs.execute(query)
curs.fetchall()

[(1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 2),
 (6, 0),
 (7, 1),
 (8, 0),
 (9, 0),
 (10, 0),
 (11, 1),
 (12, 0),
 (13, 0),
 (14, 0),
 (15, 0),
 (16, 0),
 (17, 0),
 (18, 0),
 (19, 0),
 (20, 1)]

In [15]:
#Incomplete Method 2

query = ''' SELECT cci.character_id, COUNT(cci.item_id) as weapon_count
            FROM charactercreator_character_inventory as cci, 
                 armory_weapon as aw
            WHERE cci.item_id = aw.item_ptr_id
            GROUP BY character_id
            LIMIT 20;'''
curs.execute(query)
curs.fetchall()

# 'WHERE cci.item_id = ai.item_id AND ai.item_id = aw.item_ptr_id'
# WHERE COMMAND skips zero-weapon-holders

[(5, 2),
 (7, 1),
 (11, 1),
 (20, 1),
 (22, 1),
 (23, 1),
 (26, 1),
 (27, 3),
 (29, 2),
 (30, 1),
 (32, 1),
 (34, 1),
 (35, 2),
 (36, 3),
 (37, 2),
 (38, 2),
 (39, 2),
 (40, 1),
 (41, 1),
 (47, 1)]

In [16]:
# Incomplete Method 2 Alternate Syntax

query = '''SELECT character_id, count(item_id) as weapon_count
           FROM charactercreator_character_inventory
           WHERE item_id IN
             (SELECT item_ptr_id
              FROM armory_weapon)
           GROUP BY character_id
           LIMIT 20;
        '''
curs.execute(query)
curs.fetchall()

[(5, 2),
 (7, 1),
 (11, 1),
 (20, 1),
 (22, 1),
 (23, 1),
 (26, 1),
 (27, 3),
 (29, 2),
 (30, 1),
 (32, 1),
 (34, 1),
 (35, 2),
 (36, 3),
 (37, 2),
 (38, 2),
 (39, 2),
 (40, 1),
 (41, 1),
 (47, 1)]

On average, how many Items does each Character have?

In [17]:
query = ''' SELECT AVG(item_count)
            FROM ( SELECT COUNT(id) as item_count
                   FROM charactercreator_character_inventory
                   GROUP BY character_id)
        '''
curs.execute(query)
curs.fetchall()

[(2.9735099337748343,)]

On average, how many Weapons does each character have?

In [18]:
query = ''' SELECT AVG(weapon_count)
            FROM (  
                SELECT ccc.character_id, COUNT(aw.item_ptr_id) as weapon_count
                FROM charactercreator_character as ccc
                LEFT JOIN charactercreator_character_inventory as cci                    
                ON ccc.character_id = cci.character_id
                LEFT JOIN armory_weapon as aw
                ON cci.item_id = aw.item_ptr_id
                GROUP BY ccc.character_id
            )
        '''
curs.execute(query)
curs.fetchall()

[(0.6721854304635762,)]

In [19]:
curs.close()

## Assignment - Part 2, Making and populating a Database

In [20]:
df = pd.read_csv('buddymove_holidayiq.csv')

In [21]:
conn = sqlite3.connect('buddymove_holidayiq.sqlite3')

In [22]:
df.to_sql('review', conn, if_exists='replace')

/home/martin/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [23]:
curs = conn.cursor()

In [24]:
# SQLITE - How many rows?

query = 'SELECT COUNT(*) FROM review'
curs.execute(query)
curs.fetchall()

[(249,)]

In [25]:
# DATAFRAME - How many rows?

df.shape[0]

249

In [26]:
df.head()

,User Id,Sports,Religious,Nature,Theatre,Shopping,Picnic
0,User 1,2,77,79,69,68,95
1,User 2,2,62,76,76,69,68
2,User 3,2,50,97,87,50,75
3,User 4,2,68,77,95,76,61
4,User 5,2,98,54,59,95,86


In [27]:
df[(df['Nature'] > 100) & (df['Shopping'] > 100)].head()

,User Id,Sports,Religious,Nature,Theatre,Shopping,Picnic
52,User 53,4,79,103,68,103,85
54,User 55,8,94,140,89,118,92
61,User 62,8,94,128,89,128,99
63,User 64,6,84,108,138,103,79
65,User 66,6,89,108,128,116,81


In [28]:
# SQLITE - How many users who reviewed at least 100 Nature in the category 
#          also reviewed at least 100 in the Shopping category?
query = '''SELECT COUNT(*) FROM review
           WHERE Nature > 100 AND Shopping > 100
        '''
curs.execute(query)
curs.fetchall()

[(78,)]

In [29]:
# Dataframe - How many users who reviewed at least 100 Nature in the category 
#             also reviewed at least 100 in the Shopping category?
df[(df['Nature'] > 100) & (df['Shopping'] > 100)].shape[0]

78